In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import time
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
import math
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Imputer
from sklearn.svm import SVC, OneClassSVM
from torch.utils.data import DataLoader, TensorDataset
from keras.utils import np_utils
from keras.datasets.fashion_mnist import load_data
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek
from collections import Counter
from imblearn.under_sampling import TomekLinks 
from imblearn.datasets import make_imbalance
from imblearn.metrics import classification_report_imbalanced
from sklearn.datasets import load_iris

E:\develop\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
RANDOM_STATE = 42

In [3]:
iris = load_iris()
X, y = make_imbalance(iris.data, iris.target, ratio={0: 25, 1: 50, 2: 50},
                      random_state=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE)

### missing_value

In [4]:
def fix_missing_value(data, col):
    a_mean = data[col].replace(0, np.NaN).mean()
    a_std = data[col].replace(0, np.NaN).std()
    null_count = data[col].value_counts()[0]
    min_val = a_mean - a_std
    median =data[col].replace(0, np.NaN).median()
    print(col, a_mean, a_std, min_val, median)
    a_null_random_list = [median] * null_count  # np.random.randint(min_val if min_val > 0 else 0, a_mean + a_std, size=null_count)
    data[col][data[col] == 0] = a_null_random_list
    return data

In [5]:
#对象字段
category_columns = []
# 数值字段
numeric_columns = []

encoders = {
    'categorical': {},
    'scaler': None,
    'paid': None
}

def get_data_sets(df=None,stime=None, etime=None, train = True, remove=False):
    if df is None:
        df = db_get(stime, etime)
    df = df.fillna(0)
    df['age'][df['age'] <= 20] = 1
    df['age'][(df['age'] > 20) & (df['age'] <=30)] = 2
    df['age'][(df['age'] > 30) & (df['age'] <=40)] = 3
    df['age'][df['age'] > 40] = 4
    # 成交金额分类
    df['paid'] = np.cast['int64'](np.ceil(df['src_price'] > 0))
    
    if train:
        for col in df.columns:
            if df[col].dtype != 'object' and col != 'src_price' and col != 'res_id' and col != 'paid' and col != 'buy':
                numeric_columns.append(col)
        
        encoders['scaler'] = preprocessing.MinMaxScaler()
        encoders['scaler'].fit(df[numeric_columns])
        
        for col in df.columns:
            if col not in ['src_price', 'res_id', 'paid', 'buy']:
                category_columns.append(col)
            
            if df[col].dtype == 'object':
                cc = pd.Categorical(df[col], ordered=True)
                encoders['categorical'][col] = cc.categories
                
    for c in encoders['categorical']:
        e = encoders['categorical'][c]
        cc = pd.Categorical(df[c], encoders['categorical'].get(c))
        df[c]  = cc.codes   
    if remove:
        df = df[(df.shape[1] - df.apply(lambda x: sum(x.values == 0), axis=1)) > 10]
    missing_cols = ['input_type', 'need_name', 'job', 'phone_home',
       'education', 'marriage', 'income', 'emotion_time']
    for col in missing_cols:
        df = fix_missing_value(df, col)
    feature = np.cast['float32'](df[category_columns].values)
    labels = df['paid'].values
    return (feature, labels, df)

### freq_encoding 扩展 cols 中的分类

new_columns = dataframe[col].value_counts()[category]

In [6]:
# This function late in a list of features 'cols' from train and test dataset, 
# and performing frequency encoding. 
def freq_encoding(cols, train_df, test_df):
    # we are going to store our new dataset in these two resulting datasets
    result_train_df=pd.DataFrame()
    result_test_df=pd.DataFrame()
    
    # loop through each feature column to do this
    for col in cols:
        
        # capture the frequency of a feature in the training set in the form of a dataframe
        col_freq=col+'_freq'
        freq=train_df[col].value_counts()
        freq=pd.DataFrame(freq)
        freq.reset_index(inplace=True)
        freq.columns=[[col,col_freq]]

        # merge ths 'freq' datafarme with the train data
        temp_train_df=pd.merge(train_df[[col]], freq, how='left', on=col)
        temp_train_df.drop([col], axis=1, inplace=True)

        # merge this 'freq' dataframe with the test data
        temp_test_df=pd.merge(test_df[[col]], freq, how='left', on=col)
        temp_test_df.drop([col], axis=1, inplace=True)

        # if certain levels in the test dataset is not observed in the train dataset, 
        # we assign frequency of zero to them
        temp_test_df.fillna(0, inplace=True)
        temp_test_df[col_freq]=temp_test_df[col_freq].astype(np.int32)

        if result_train_df.shape[0]==0:
            result_train_df=temp_train_df
            result_test_df=temp_test_df
        else:
            result_train_df=pd.concat([result_train_df, temp_train_df],axis=1)
            result_test_df=pd.concat([result_test_df, temp_test_df],axis=1)
    
    return result_train_df, result_test_df

### ###  binary_encoding 扩展 feat 中的分类
max_bin_len = len("{0:b}".format(max_value))

x = "{0:b}".format(35).zfill(max_bin_len)

new_column = pd.Series(list(x)).astype(np.uint8)

In [7]:
# perform binary encoding for categorical variable
# this function take in a pair of train and test data set, and the feature that need to be encode.
# it returns the two dataset with input feature encoded in binary representation
# this function assumpt that the feature to be encoded is already been encoded in a numeric manner 
# ranging from 0 to n-1 (n = number of levels in the feature). 

def binary_encoding(train_df, test_df, feat):
    # calculate the highest numerical value used for numeric encoding
    train_feat_max = train_df[feat].max()
    test_feat_max = test_df[feat].max()
    if train_feat_max > test_feat_max:
        feat_max = train_feat_max
    else:
        feat_max = test_feat_max

    # use the value of feat_max+1 to represent missing value
    train_df.loc[train_df[feat] == 0, feat] = feat_max + 1
    test_df.loc[test_df[feat] == 0, feat] = feat_max + 1
    train_df.loc[train_df[feat] == -1, feat] = feat_max + 1
    test_df.loc[test_df[feat] == -1, feat] = feat_max + 1

    # create a union set of all possible values of the feature
    union_val = np.union1d(train_df[feat].unique(), test_df[feat].unique())
    # extract the highest value from from the feature in decimal format.
    max_dec = union_val.max()

    # work out how the ammount of digtis required to be represent max_dev in binary representation
    max_bin_len = len("{0:b}".format(max_dec))
    index = np.arange(len(union_val))
    columns = list([feat])
    
    # create a binary encoding feature dataframe to capture all the levels for the feature
    bin_df = pd.DataFrame(index=index, columns=columns)
    bin_df[feat] = union_val
    
    # capture the binary representation for each level of the feature 
    feat_bin = bin_df[feat].apply(lambda x: "{0:b}".format(x).zfill(max_bin_len))
    
    # split the binary representation into different bit of digits 
    splitted = feat_bin.apply(lambda x: pd.Series(list(x)).astype(np.uint8))
    splitted.columns = [feat + '_bin_' + str(x) for x in splitted.columns]
    bin_df = bin_df.join(splitted)
    
    # merge the binary feature encoding dataframe with the train and test dataset - Done! 
    train_df = pd.merge(train_df, bin_df, how='left', on=[feat])
    test_df = pd.merge(test_df, bin_df, how='left', on=[feat])
    return train_df, test_df

In [8]:
cr = Counter(y_train)

In [9]:
print(sorted(Counter(y_train).items()))

[(0, 17), (1, 38), (2, 38)]


## ## 2. K-fold CV with Out-of-Fold Prediction

### 2.1 OOF utility functions

In [10]:
def auc_to_gini_norm(auc_score):
    return 2*auc_score-1

In [11]:
n_folds=3
skf=StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=2018)

### 2.1.1 Sklearn K-fold & OOF function

In [37]:
def cross_validate_sklearn(clf, x_train, y_train, x_test, y_test, kf, scale=False, verbose=True, ruc=False):
    start_time = time.time()
    
    # initialise the size of out-of-fold train an test prediction
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))
    train_pred_prob = np.zeros((x_train.shape[0]))
    dataset_blend_test_j = np.zeros((x_test.shape[0], n_folds))
    for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
        x_train_kf, x_val_kf = x_train[train_index], x_train[test_index]
        y_train_kf, y_val_kf = y_train[train_index], y_train[test_index]
        
        clf.fit(x_train_kf, y_train_kf)
        train_pred[test_index] = clf.predict(x_val_kf)
        train_pred_prob[test_index] = clf.predict(x_val_kf)
        dataset_blend_test_j[:, i] = clf.predict(x_test)
    test_pred = dataset_blend_test_j.sum(axis=1) // (n_folds //2 + 1)
    return train_pred, test_pred, train_pred_prob

### 2.1.2 Xgboost K-fold & OOF function

In [54]:
def probability_to_rank(prediction, scaler=1):
    pred_df=pd.DataFrame(columns=['probability'])
    pred_df['probability']=prediction
    pred_df['rank']=pred_df['probability'].rank()/len(prediction)*scaler
    return pred_df['rank'].values

In [58]:
def cross_validate_xgb(params, x_train, y_train, x_test, y_test, kf, verbose=True,
                      verbose_eval=50, num_boost_round=4000, use_rank=True):
    start_time = time.time()
    
    # initialise the size of out-of-fold train an test prediction
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))
    dataset_blend_test_j = np.zeros((x_test.shape[0], n_folds))
    for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
        x_train_kf, x_val_kf = x_train[train_index], x_train[test_index]
        y_train_kf, y_val_kf = y_train[train_index], y_train[test_index]
        d_train_kf = xgb.DMatrix(x_train_kf, y_train_kf)
        d_val_kf = xgb.DMatrix(x_val_kf, y_val_kf)
        d_test = xgb.DMatrix(x_test, y_test)
        
        bst = xgb.train(params, d_train_kf, num_boost_round=num_boost_round)
        train_pred[test_index] = bst.predict(d_val_kf, ntree_limit=bst.best_ntree_limit)
        dataset_blend_test_j[:, i] = bst.predict(d_test)
        # test_pred+=probability_to_rank(bst.predict(d_test))
    # test_pred /= kf.n_splits
    test_pred = dataset_blend_test_j.sum(axis=1) // (n_folds //3 + 1)
    return train_pred, test_pred

### 2.1.3 LigthGBM K-fold & OOF function

In [39]:
def cross_validate_lgb(params, x_train, y_train, x_test, kf,
                       verbose=True, verbose_eval=50, use_cat=True, use_rank=True):
    start_time = time.time()
    
    # initialise the size of out-of-fold train an test prediction
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))
    train_pred_prob = np.zeros((x_train.shape[0]))
    dataset_blend_test_j = np.zeros((x_test.shape[0], n_folds))
    for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
        x_train_kf, x_val_kf = x_train[train_index], x_train[test_index]
        y_train_kf, y_val_kf = y_train[train_index], y_train[test_index]
        lgb_train = lgb.Dataset(np.array(x_train_kf), y_train_kf)
        lgb_val = lgb.Dataset(np.array(x_val_kf), y_val_kf, reference=lgb_train)
        
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=4000,
                        valid_sets=lgb_val,
                        early_stopping_rounds=30
                    )
        train_pred[test_index] = np.argmax(gbm.predict(x_val_kf), axis=1)
        dataset_blend_test_j[:, i] =  np.argmax(gbm.predict(x_test), axis=1)
    test_pred = dataset_blend_test_j.sum(axis=1) // (n_folds //2 + 1)
    if verbose:
        print('xgb train:{}'.format(classification_report(y_train, train_pred)))
        print('xgb test:{}'.format(classification_report(y_test, test_pred)))
    return train_pred, test_pred

## 3. Generate level 1 OOF predictions

### 3.1 random forest

In [40]:
rf = RandomForestClassifier(n_estimators=200, n_jobs=6, max_depth=7,
                           criterion='entropy', random_state=2018)
rf_train_pred, rf_test_pred, _ = cross_validate_sklearn(rf, x_train=X_train, y_train=y_train,x_test=X_test, y_test=y_test, kf=skf,scale=False, verbose=True, ruc=False)

### 3.2 Extra Tree

In [41]:
et = ExtraTreesClassifier(n_estimators=100, criterion="gini", max_depth=5,
                          min_samples_split=5, n_jobs=-1, random_state=1)
et_train_pred, et_test_pred, _ = cross_validate_sklearn(et, x_train=X_train, y_train=y_train,x_test=X_test, y_test=y_test, kf=skf,scale=False, verbose=True)

### 3.3 Logistic Regression

In [42]:
logit=LogisticRegression(random_state=0, C=0.5,class_weight={0:1, 1:1})
logit_train_pred, logit_test_pred, train_pred_prob = cross_validate_sklearn(logit, x_train=X_train, y_train=y_train,x_test=X_test, y_test=y_test, kf=skf,scale=False, verbose=True)

### 3.4 BernoulliNB

In [43]:
nb=BernoulliNB()
nb_train_pred, nb_test_pred, _ = cross_validate_sklearn(nb, x_train=X_train, y_train=y_train,x_test=X_test, y_test=y_test, kf=skf,scale=False, verbose=True)

### 3.5 XGB

In [59]:
xgb_params = {
    "objective" :  'multi:softmax',
    "tree_method": "hist",
    "eval_metric": "auc",
    "eta": 0.1,
    "max_depth": 5,
    "min_child_weight": 10,
    "gamma": 0.70,
    "subsample": 0.76,
    "colsample_bytree": 0.95,
    "nthread": 6,
    "seed": 0,
    'silent': 1,
    'num_class': 3,
}

xgb_train_pred, xgb_test_pred = cross_validate_xgb(xgb_params, X_train, y_train, X_test,y_test, skf, use_rank=False, verbose_eval=False)

### 3.7 LightGBM

In [61]:
lgb_params = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'softmax',
    'metric': {'auc'},
    'num_leaves': 22,
    'min_sum_hessian_in_leaf': 20,
    'max_depth': 5,
    'learning_rate': 0.1,  # 0.618580
    'num_threads': 6,
    'feature_fraction': 0.6894,
    'bagging_fraction': 0.4218,
    'max_drop': 5,
    'drop_rate': 0.0023,
    'min_data_in_leaf': 10,
    'bagging_freq': 1,
    'lambda_l1': 1,
    'lambda_l2': 0.01,
    'verbose': 1,
    'num_class': 3
}


cat_cols=[]
lgm_train_pred, lgm_test_pred = cross_validate_lgb(lgb_params,X_train, y_train ,X_test, y_test, skf, use_cat=True, 
                            verbose_eval=False, use_rank=False)

[1]	valid_0's auc: 0.5
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's auc: 0.5
[3]	valid_0's auc: 0.5
[4]	valid_0's auc: 0.5
[5]	valid_0's auc: 0.5
[6]	valid_0's auc: 0.5
[7]	valid_0's auc: 0.5
[8]	valid_0's auc: 0.5
[9]	valid_0's auc: 0.5
[10]	valid_0's auc: 0.5
[11]	valid_0's auc: 0.5
[12]	valid_0's auc: 0.5
[13]	valid_0's auc: 0.5
[14]	valid_0's auc: 0.5
[15]	valid_0's auc: 0.5
[16]	valid_0's auc: 0.5
[17]	valid_0's auc: 0.5
[18]	valid_0's auc: 0.5
[19]	valid_0's auc: 0.5
[20]	valid_0's auc: 0.5
[21]	valid_0's auc: 0.5
[22]	valid_0's auc: 0.5
[23]	valid_0's auc: 0.5
[24]	valid_0's auc: 0.5
[25]	valid_0's auc: 0.5
[26]	valid_0's auc: 0.5
[27]	valid_0's auc: 0.5
[28]	valid_0's auc: 0.5
[29]	valid_0's auc: 0.5
[30]	valid_0's auc: 0.5
[31]	valid_0's auc: 0.5
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
[1]	valid_0's auc: 0.5
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's auc: 0.5
[3]	valid_0's auc: 0.5
[4]	valid_0's 

E:\develop\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 4. Level 2 ensemble

### 4.1 Generate L1 output dataframe

In [62]:
lv1_train = np.stack((rf_train_pred, et_train_pred, logit_train_pred, nb_train_pred, xgb_train_pred, lgm_train_pred), axis=-1)
lv1_test = np.stack((rf_test_pred, et_test_pred, logit_test_pred, nb_test_pred, xgb_test_pred, lgm_test_pred), axis=-1)

### 4.2 Level 2 XGB

In [64]:
xgb_params = {
    "objective" :  'multi:softmax',
    "tree_method": "hist",
    "eval_metric": "auc",
    "eta": 0.1,
    "max_depth": 5,
    "min_child_weight": 10,
    "gamma": 0.70,
    "subsample": 0.76,
    "colsample_bytree": 0.95,
    "nthread": 6,
    "seed": 0,
    'silent': 1,
    'num_class': 3
}

lv2_xgb_train_pred, lv2_xgb_test_pred = cross_validate_xgb(xgb_params, lv1_train, y_train, lv1_test,y_test, skf, use_rank=False, verbose_eval=False)

### 4.3 Logistic Regression

In [66]:
clf = LogisticRegression()
clf.fit(lv1_train, y_train)  
prediction = clf.predict(lv1_test)

In [67]:
print(classification_report(y_test, prediction))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         8
          1       0.92      0.92      0.92        12
          2       0.92      0.92      0.92        12

avg / total       0.94      0.94      0.94        32

